In [1]:
%%writefile C:/Users/ACER/AppData/Roaming/jupyter/kernels/going_modular/get_data.py
import os
import zipfile

from pathlib import Path

import requests

def get_data(data_path: str,
             file_name: str,
             file_dir_or_url: str):


    # Setup path to data folder
    data_path = Path(data_path)
    image_path = data_path / file_name.removesuffix('.zip')

    # If the image folder doesn't exist, download it and prepare it... 
    if image_path.is_dir():
        print(f"{image_path} directory exists.")
    else:
        print(f"Did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)

    # Download pizza, steak, sushi data
    with open(data_path / file_name, "wb") as f:
        request = requests.get(file_dir_or_url)
        print(f"Downloading {file_name} data...")
        f.write(request.content)

    # Unzip pizza, steak, sushi data
    with zipfile.ZipFile(data_path / file_name, "r") as zip_ref:
        print(f"Unzipping {file_name} data...") 
        zip_ref.extractall(image_path)

    # Remove zip file
    os.remove(data_path / file_name)

Overwriting C:/Users/ACER/AppData/Roaming/jupyter/kernels/going_modular/get_data.py


In [2]:
%%writefile C:/Users/ACER/AppData/Roaming/jupyter/kernels/going_modular/data_setup.py
"""
Contains functionalitiy for creating Pytorch Datalaoder's for 
image classification
"""
import os
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

NUM_WORKER = os.cpu_count()


def Create_datalaoder(
    train_dir: str,
    test_dir: str,
    transform: transforms.Compose,
    batch_size: int,
    num_workers: int = NUM_WORKER  
):
    """
    Create train and testing dataloaders.
    
      Takes in a training directory and testing directroy path and turns them into 
      PyTorch Datasets and then into PyTorch DataLoaders.

      Args:
        train_dir: Path to training directory.
        test_dir: Path to testing directory.
        transform: torchvision transforms to perform on training and testing data.
        batch_size: Number of samples per batch in each of the DataLoaders.
        num_workers: An integer for number of workers per DataLoader.

      Returns:
        A tuple of (train_dataloader, test_dataloader, class_names).
        Where class_names is a list of the target classes.
        Example usage:
          train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=path/to/train_dir,
            test_dir=path/to/test_dir,
            transform=some_transform,
            batch_size=32,
            num_workers=4)
  """
    # Creating train and test datasets.
    train_data = torchvision.datasets.ImageFolder(root = train_dir,
                                                  transform = transform)
    test_data = torchvision.datasets.ImageFolder(root = test_dir,
                                                 transform = transform)
    # Creating Dataloader
    train_datalaoder = DataLoader(dataset = train_data,
                                  batch_size = batch_size,
                                  shuffle = True)
    test_datalaoder = DataLoader(dataset = test_data,
                                 batch_size = batch_size,
                                 shuffle = False)
    
    return train_datalaoder, test_datalaoder, train_data.classes
    

Overwriting C:/Users/ACER/AppData/Roaming/jupyter/kernels/going_modular/data_setup.py


In [3]:
%%writefile C:/Users/ACER/AppData/Roaming/jupyter/kernels/going_modular/model_builder.py

"""
Create a tinyVGG model.
Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
See the original architecture here: https://poloclub.github.io/cnn-explainer/
  
  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
"""
from torch import nn

class TinyVGG(nn.Module):
    
    def __init__(self,input_shape: int, hidden_units: int, output_shape: int) -> None:

        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels = input_shape,
                      out_channels = hidden_units,
                      kernel_size = 3,
                      stride = 1, padding = 0),
            nn.ReLU(),
            nn.Conv2d(in_channels = hidden_units,
                      out_channels = hidden_units,
                      kernel_size = 3,
                      stride = 1,
                      padding = 0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2,
                         stride = 2)

        )
        self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels = hidden_units,
                  out_channels = hidden_units,
                  kernel_size = 3,
                  stride = 1, padding = 0),
        nn.ReLU(),
        nn.Conv2d(in_channels = hidden_units,
                  out_channels = hidden_units,
                  kernel_size = 3,
                  stride = 1,
                  padding = 0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2,
                      stride = 2)

        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features = hidden_units * 169,
                      out_features = output_shape)
        )

    def forward(self, x):
        return self.classifier(self.conv_block_2(self.conv_block_1(x)))




Overwriting C:/Users/ACER/AppData/Roaming/jupyter/kernels/going_modular/model_builder.py


In [4]:
%%writefile C:/Users/ACER/AppData/Roaming/jupyter/kernels/going_modular/engine.py
"""
Contains functions for training and testing a PyTorch model.
"""
from typing import Dict, List, Tuple

import torch

from tqdm.auto import tqdm

def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    
    """Trains a PyTorch model for a single epoch.

    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).

    Args:
      model: A PyTorch model to be trained.
      dataloader: A DataLoader instance for the model to be trained on.
      loss_fn: A PyTorch loss function to minimize.
      optimizer: A PyTorch optimizer to help minimize the loss function.
      device: A target device to compute on (e.g. "cuda" or "cpu").

    Returns:
      A tuple of training loss and training accuracy metrics.
      In the form (train_loss, train_accuracy). For example:

      (0.1112, 0.8743)
    """
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        

        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

          # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

        # Adjust metrics to get average loss and accuracy per batch 
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    
    """Tests a PyTorch model for a single epoch.

      Turns a target PyTorch model to "eval" mode and then performs
      a forward pass on a testing dataset.

      Args:
        model: A PyTorch model to be tested.
        dataloader: A DataLoader instance for the model to be tested on.
        loss_fn: A PyTorch loss function to calculate loss on the test data.
        device: A target device to compute on (e.g. "cuda" or "cpu").

      Returns:
        A tuple of testing loss and testing accuracy metrics.
        In the form (test_loss, test_accuracy). For example:

        (0.0223, 0.8985)
      """
  # Put model in eval mode
    model.eval() 
  
  # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0
  
  # Turn on inference context manager
    with torch.inference_mode():
        
      # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            
          # Send data to target device
            X, y = X.to(device), y.to(device)
  
          # 1. Forward pass
            test_pred_logits = model(X)

          # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()
          
          # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
          
  # Adjust metrics to get average loss and accuracy per batch 
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc


def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List[float]]:
    
    """Trains and tests a PyTorch model.

      Passes a target PyTorch models through train_step() and test_step()
      functions for a number of epochs, training and testing the model
      in the same epoch loop.

      Calculates, prints and stores evaluation metrics throughout.

      Args:
        model: A PyTorch model to be trained and tested.
        train_dataloader: A DataLoader instance for the model to be trained on.
        test_dataloader: A DataLoader instance for the model to be tested on.
        optimizer: A PyTorch optimizer to help minimize the loss function.
        loss_fn: A PyTorch loss function to calculate loss on both datasets.
        epochs: An integer indicating how many epochs to train for.
        device: A target device to compute on (e.g. "cuda" or "cpu").

      Returns:
        A dictionary of training and testing loss as well as training and
        testing accuracy metrics. Each metric has a value in a list for 
        each epoch.
        In the form: {train_loss: [...],
                      train_acc: [...],
                      test_loss: [...],
                      test_acc: [...]} 
        For example if training for epochs=2: 
                     {train_loss: [2.0616, 1.0537],
                      train_acc: [0.3945, 0.3945],
                      test_loss: [1.2641, 1.5706],
                      test_acc: [0.3400, 0.2973]} 
      """
  # Create empty results dictionary
    results = {"train_loss": [],
        "train_acc": [],
        "test_loss": [],
        "test_acc": []      
        }
  
  # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)
      
      # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
      )

      # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
    return results

Overwriting C:/Users/ACER/AppData/Roaming/jupyter/kernels/going_modular/engine.py


In [5]:
%%writefile C:/Users/ACER/AppData/Roaming/jupyter/kernels/going_modular/utils.py
"""
File containing various utility functions for PyTorch model training.
""" 
import torch

from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):

    """Saves a PyTorch model to a target directory.

      Args:
        model: A target PyTorch model to save.
        target_dir: A directory for saving the model to.
        model_name: A filename for the saved model. Should include
          either ".pth" or ".pt" as the file extension.

      Example usage:
        save_model(model=model_0,
                   target_dir="models",
                   model_name="05_going_modular_tingvgg_model.pth")
      """
      # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                            exist_ok=True)

      # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

      # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
               f=model_save_path)

Overwriting C:/Users/ACER/AppData/Roaming/jupyter/kernels/going_modular/utils.py


In [6]:
%%writefile C:/Users/ACER/AppData/Roaming/jupyter/kernels/going_modular/train.py

def train(URL: str,
          data_path: str,
          file_name: str,
          model_name: str
          
    

):
    # load or Download data
    # Create dataloader using data_setup
    # build the model using model builder
    # start training with engine
    # save model with utils
    """
    Trains a PyTorch image classification model using device-agnostic code.
    Be aware that the data should be a zip file which contains a file and this file 
    contains two files nemed train and test 
    """
    import os
    import torch

    from torchvision import transforms
    from timeit import default_timer as timer 
    from going_modular import data_setup, engine, model_builder, utils, get_data
    # import data_setup, engine, model_builder, utils

    # Setup hyperparameters
    NUM_EPOCHS = 5 
    BATCH_SIZE = 32
    HIDDEN_UNITS = 10
    LEARNING_RATE = 0.001

    # data_path = "C:/Users/ACER/AppData/Roaming/jupyter/kernels/going_modular/data"
    # file_name = 'pizza_steak_sushi.zip'
    # file_dir_or_url = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip"
    
    # load or Download data
    Download_data = get_data.get_data(data_path= data_path,
                             file_name = file_name,
                             file_dir_or_url = URL)
    
    filename_withoutsuffix = str(file_name.removesuffix('.zip'))
    # Setup directories
    train_dir = data_path + filename_withoutsuffix + '/train'
    test_dir = data_path + filename_withoutsuffix + '/test'

    # Setup device agnostic code
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Create transforms
    data_transform = transforms.Compose([
                                         transforms.Resize((64, 64)),
                                         transforms.ToTensor()
    ])

    # Create DataLoader's and get class_names
    train_dataloader, test_dataloader, class_names = data_setup.Create_datalaoder(train_dir=train_dir, 
                                                                                   test_dir=test_dir,
                                                                                   transform=data_transform,
                                                                                   batch_size=BATCH_SIZE)

    # Create model
    model = model_builder.TinyVGG(input_shape=3,
                                  hidden_units=HIDDEN_UNITS,
                                  output_shape=len(class_names)).to(device)

    # Setup loss and optimizer
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=LEARNING_RATE)

    # Start the timer

    start_time = timer()

    # Start training with help from engine.py
    engine.train(model=model,
                 train_dataloader=train_dataloader,
                 test_dataloader=test_dataloader,
                 loss_fn=loss_fn,
                 optimizer=optimizer,
                 epochs=NUM_EPOCHS, 
                 device=device)

    # End the timer and print out how long it took
    end_time = timer()
    print(f"[INFO] Total training time: {end_time-start_time:.3f} seconds")

    # Save the model to file
    utils.save_model(model=model,
                     target_dir='C:/Users/ACER/AppData/Roaming/jupyter/kernels/going_modular/models',
                     model_name= model_name)



Overwriting C:/Users/ACER/AppData/Roaming/jupyter/kernels/going_modular/train.py


In [7]:
cd "C:/Users/ACER/AppData/Roaming/jupyter/kernels/

C:\Users\ACER\AppData\Roaming\jupyter\kernels


In [8]:
from going_modular import train
train.train(URL = "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
            data_path="C:/Users/ACER/AppData/Roaming/jupyter/kernels/going_modular/data/",
            file_name='pizza_steak_sushi.zip',
            model_name='model_1.pth')

C:\Users\ACER\AppData\Roaming\jupyter\kernels\going_modular\data\pizza_steak_sushi directory exists.
Unzipping pizza_steak_sushi.zip data...


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0977 | train_acc: 0.3984 | test_loss: 1.1045 | test_acc: 0.1979
Epoch: 2 | train_loss: 1.0846 | train_acc: 0.4141 | test_loss: 1.1243 | test_acc: 0.1979
Epoch: 3 | train_loss: 1.1011 | train_acc: 0.2930 | test_loss: 1.1364 | test_acc: 0.1979
Epoch: 4 | train_loss: 1.0417 | train_acc: 0.5078 | test_loss: 1.0949 | test_acc: 0.3229
Epoch: 5 | train_loss: 0.9740 | train_acc: 0.5547 | test_loss: 1.0910 | test_acc: 0.2812
[INFO] Total training time: 19.939 seconds
[INFO] Saving model to: C:\Users\ACER\AppData\Roaming\jupyter\kernels\going_modular\models\model_1.pth


In [ ]:
# works that should be done in future
# 1 _ add an option to set the epochs
# 2 _ add an option to set the models like resnet and ...
# 3 _ add an option to set loss_fn and optimizer
# 4 _ add options to set BATCH_SIZE HIDDEN_UNITS LEARNING_RATE
# 5 _ add option to set transforms
# 6 _ add option to set manual seed
# 7 _ make it visualize train loss and test loss and train acc and test acc with matplotlib
# 8 _ add ao option to set where model should be seved
# 9 _ add an option to use a model that trained befor
# 10 _ make it plot loss curves and acc curves(you can use plot curve function in helper_function)
# 11 _ ...

In [ ]:
# کافیه دایرکتوری رو به آدرس زیر تغییر بدی و ماژول ترین رو وارد کنی
# cd "C:/Users/ACER/AppData/Roaming/jupyter/kernels/
# from going_modular import train